In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uzbek-language-dataset-sentence-and-audio/dataset_dict.json
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00001-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/state.json
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00006-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/dataset_info.json
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00007-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00008-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00005-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00002-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00003-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00004-of-00010.arrow
/kaggle/input/uzbek-language-dataset-sentence-and-audio/test/data-00009-of-0001

In [2]:
from datasets import load_from_disk

common_voice_train = load_from_disk("/kaggle/input/uzbek-language-dataset-sentence-and-audio/train")
common_voice_test = load_from_disk("/kaggle/input/uzbek-language-dataset-sentence-and-audio/test")
print(common_voice_train)
print(common_voice_test)

Loading dataset from disk:   0%|          | 0/82 [00:00<?, ?it/s]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 42242
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 4928
})


In [3]:
sample = common_voice_train[65]

# Input features
input_feats = sample["input_features"]
print("Input shape:", len(input_feats), "x", len(input_feats[0]))  # (time_steps x 80)

# Label
print("Label:", sample["labels"])
print("Label length:", len(sample["labels"]))


Input shape: 80 x 3000
Label: [50258, 50337, 50359, 50363, 8524, 73, 345, 20291, 4116, 75, 1083, 72, 10806, 4097, 71, 9229, 72, 277, 2282, 270, 559, 27442, 7852, 2604, 89, 742, 14697, 5843, 50257]
Label length: 29


In [4]:
sample = common_voice_test[66]

# Input features
input_feats = sample["input_features"]
print("Input shape:", len(input_feats), "x", len(input_feats[0]))  # (time_steps x 80)

# Label
print("Label:", sample["labels"])
print("Label length:", len(sample["labels"]))

Input shape: 80 x 3000
Label: [50258, 50337, 50359, 50363, 4575, 3015, 4842, 8206, 10072, 1299, 1371, 514, 50257]
Label length: 13


In [5]:
!pip install transformers datasets evaluate accelerate bitsandbytes peft
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118  # Install GPU-supported PyTorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.5 MB/s eta 0:00:00:00:0100:01
Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
import torch
print(torch.cuda.is_available())  # Should return True

True


In [7]:
model_name_or_path = "openai/whisper-small"
language = "Uzbek"
language_abbr = "uz"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_13_0"

In [8]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [9]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [10]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

In [11]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Pad input features
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Convert labels to tensors and pad them
        labels = [torch.tensor(feature["labels"], dtype=torch.long) for feature in features]
        labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=self.processor.tokenizer.pad_token_id)

        # Replace padding tokens (50257) with -100 to ignore them in loss calculation
        labels_padded = labels_padded.masked_fill(labels_padded == self.processor.tokenizer.pad_token_id, -100)

        # If all labels start with BOS token, remove it
        bos_token_id = self.processor.tokenizer.bos_token_id
        if bos_token_id is not None and (labels_padded[:, 0] == bos_token_id).all().item():
            labels_padded = labels_padded[:, 1:]

        batch["labels"] = labels_padded
        return batch


In [12]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    #max_label_length=448  # Can be adjusted
)

In [13]:
!pip install -U bitsandbytes
!pip install accelerate

In [14]:
from transformers import WhisperForConditionalGeneration
import torch

if torch.cuda.is_available():
	model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")
else:
	model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [15]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model, PeftModel, LoraModel

model = prepare_model_for_kbit_training(model)

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.4429


In [16]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [17]:
import os
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import WhisperForConditionalGeneration

 

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)
model = model.to("cuda")  # Move model to GPU

# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = torch.nn.DataParallel(model) 

In [18]:
common_voice_train = common_voice_train.select(range(42000))
common_voice_test = common_voice_test.select(range(4200))
common_voice_train, common_voice_test

(Dataset({
     features: ['input_features', 'labels'],
     num_rows: 42000
 }),
 Dataset({
     features: ['input_features', 'labels'],
     num_rows: 4200
 }))

In [19]:
import os
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import WhisperForConditionalGeneration

# ✅ Define training arguments
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-finetuned",  # Directory to save checkpoints
    per_device_train_batch_size=4,  # Small batch size for training
    per_device_eval_batch_size=4,  # Smaller batch size for evaluation
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    learning_rate=1e-4,  # Learning rate
    warmup_steps=50,  # Warmup steps for learning rate scheduler
    num_train_epochs=2,  # Number of training epochs
    save_strategy="epoch",  # Save at the end of each epoch
    save_total_limit=2,  # Keep only the last 2 checkpoints
    evaluation_strategy="steps",  # Evaluate every N steps
    eval_steps=500,
    logging_steps=100,  # Log training metrics every 100 steps
    generation_max_length=128,  # Maximum length for generation during evaluation
    fp16=True,  # Use mixed precision to reduce memory usage
    remove_unused_columns=False,  # Remove unused columns from the dataset to save memory
    optim="adamw_bnb_8bit",  # Use 8-bit AdamW optimizer to reduce memory usage
    report_to="none", # Disable reporting to external services (e.g., Weights & Biases)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
from transformers import Seq2SeqTrainer
# ✅ Create trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_train,  # Your dataset
    eval_dataset=common_voice_test,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor
)

<ipython-input-20-0808186943d6>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [24]:
!nvidia-smi

Sun Apr  6 05:15:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             29W /   70W |    9197MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
common_voice_train, common_voice_test

(Dataset({
     features: ['input_features', 'labels'],
     num_rows: 42000
 }),
 Dataset({
     features: ['input_features', 'labels'],
     num_rows: 4200
 }))

In [23]:
trainer.train()  # Start training from scratch

# Define the directory to save the final model
final_model_dir = "./whisper-small-finetuned"

# Save the fine-tuned model and processor
trainer.save_model(final_model_dir)
processor.save_pretrained(final_model_dir)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.588400,0.596903
1000,0.428100,0.480948
1500,0.196700,0.416555
2000,0.170100,0.374123
2500,0.145300,0.335988


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimen

[]

### Pushing to Hugging Face repository

In [30]:
!pip install -U huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 9.6 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.29.0
    Uninstalling huggingface-hub-0.29.0:
      Successfully uninstalled huggingface-hub-0.29.0


In [28]:
from huggingface_hub import notebook_login
notebook_login()  # Enter your token when prompted

In [32]:
from huggingface_hub import HfApi, HfFolder

# 🔐 Paste your write token here (keep it secret!)
hf_token = "........................."

# Save the token to use with huggingface_hub
HfFolder.save_token(hf_token)

# Test if login works
api = HfApi()
print("Logged in as:", api.whoami()["name"])


Logged in as: javokhirraimov


In [34]:
from huggingface_hub import create_repo, upload_folder

repo_id = "javokhirraimov/uzbek-stt-whisper-small"
local_path = "/kaggle/working/whisper-small-finetuned"

# Create and upload
create_repo(repo_id=repo_id, exist_ok=True)
upload_folder(
    repo_id=repo_id,
    folder_path=local_path,
    commit_message="Upload fine-tuned Whisper Small model for Uzbek STT",
)


CommitInfo(commit_url='https://huggingface.co/javokhirraimov/uzbek-stt-whisper-small/commit/f884cb27635db55f8a21d9e2e988acce01a1d016', commit_message='Upload fine-tuned Whisper Small model for Uzbek STT', commit_description='', oid='f884cb27635db55f8a21d9e2e988acce01a1d016', pr_url=None, repo_url=RepoUrl('https://huggingface.co/javokhirraimov/uzbek-stt-whisper-small', endpoint='https://huggingface.co', repo_type='model', repo_id='javokhirraimov/uzbek-stt-whisper-small'), pr_revision=None, pr_num=None)

### Test the model

In [25]:
# prompt: lets test my model with examlpe from my test data
# Get a sample from the test dataset
first_sample = common_voice_test[1200]

# Prepare the input for the model
input_features = torch.tensor(first_sample["input_features"]).unsqueeze(0).to("cuda")

# Generate prediction using the model
predicted_ids = model.generate(input_features)

# Decode the prediction
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Get the actual label
label = processor.decode(first_sample["labels"], skip_special_tokens=True)

print("Predicted transcription:", transcription)
print("Actual transcription:", label)

# Compute WER or other metrics (optional)
# wer = metric.compute(predictions=[transcription], references=[label])
# print("WER:", wer)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Predicted transcription: Kozga aytaman qari quloqqa eshit ogizga gapirma                                                                                                                                                                                                                                                                                                                                                                                                                                       
Actual transcription:  Kozga aytaman Qara quloqqa Eshit ogizga Gapirma


In [27]:
first_sample = common_voice_test[1000]

# Preprocessing for model
input_features = torch.tensor(first_sample["input_features"]).unsqueeze(0).to("cuda")

# Model generations config
predicted_ids = model.generate(
    input_features,
    max_length=100,
    repetition_penalty=2.0,
    no_repeat_ngram_size=3,
    length_penalty=0.8,
    num_beams=5,
    early_stopping=True
)

# decoding  transcription 
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Haqiqiy transkripsiyani olish
label = processor.decode(first_sample["labels"], skip_special_tokens=True)

print("Predicted transcription:", transcription)
print("Actual transcription:", label)

# WER yoki boshqa metrikalarni hisoblash (ixtiyoriy)
# wer = metric.compute(predictions=[transcription], references=[label])
# print("WER:", wer)

Predicted transcription: Judayam qattiqqavozdan taraxt bolib qolgan edi  deb oylaydi tajribatida shunday emas qildi qovozidan taraqd bolib kongan edi ijda yordamlashadi tarixga boladigan edi aylab qoyilmagan edi uchta yetkazilmasliklari bnida
Actual transcription: judayam qattiq ovozdan karaxt bolib qolgan edi


In [58]:
first_sample = common_voice_test[117]

# Preprocessing for model
input_features = torch.tensor(first_sample["input_features"]).unsqueeze(0).to("cuda")

# Model generations config
predicted_ids = model.generate(
    input_features,
    max_length=100,
    repetition_penalty=2.0,
    no_repeat_ngram_size=3,
    length_penalty=0.8,
    num_beams=5,
    early_stopping=True
)

# decoding  transcription 
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Haqiqiy transkripsiyani olish
label = processor.decode(first_sample["labels"], skip_special_tokens=True)

print("Predicted transcription:", transcription)
print("Actual transcription:", label)

# WER yoki boshqa metrikalarni hisoblash (ixtiyoriy)
# wer = metric.compute(predictions=[transcription], references=[label])
# print("WER:", wer)

Predicted transcription: Samarqand shahri kochalardan ulkan bayroq golib otildi  deb oshirilgan ham mumtozalarni bayrog qolib otirdi Oylayotgani olamizda bolayotgachalaridan ulkanbayroq olib yotildi Otildi Oylangizmi bilan kuchalardan uygan bayrogga olib otilib otilding
Actual transcription: Samarqand shahri kochalardan ulkan bayroq olib otildi
